In [6]:
import pandas as pd

df = pd.read_csv("generated_essays_merged.csv")
mtld = pd.read_csv("mtld_results.csv")

In [7]:
import nltk
from lexical_diversity import lex_div as ld
from scipy import stats
import numpy as np
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\strow\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\strow\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [8]:
from lexical_diversity import lex_div as ld
from scipy import stats

# NLTK 데이터 다운로드
nltk.data.find('tokenizers/punkt')

def preprocess_for_lexdiv(text):
    #정제된 단어 리스트 만들기
    tokens = nltk.word_tokenize(str(text).lower())
    words = [word for word in tokens if word.isalpha()] # 구두점, 숫자 제거
    return words

MTLD_scores = pd.DataFrame()

print("MTLD 계산을 위해 텍스트 전처리 중...")
MTLD_scores['DPO'] = df['HumanLLMs/Human-Like-LLama3-8B-Instruct_Generated_Essay'].apply(preprocess_for_lexdiv)
MTLD_scores['CFT'] = df['meta-llama/Meta-Llama-3-8B-Instruct_Generated_Essay'].apply(preprocess_for_lexdiv)
MTLD_scores['Human'] = df['Human_Written_Essay'].apply(preprocess_for_lexdiv)

print("MTLD 점수 계산 중... (행이 많으면 시간이 걸릴 수 있습니다)")

# MTLD 계산
MTLD_scores['DPO_mtld'] = MTLD_scores['DPO'].apply(ld.mtld)
MTLD_scores['CFT_mtld'] = MTLD_scores['CFT'].apply(ld.mtld)
MTLD_scores['Human_mtld'] = MTLD_scores['Human'].apply(ld.mtld)

MTLD_scores['mtld_diff'] = MTLD_scores['DPO_mtld'] - MTLD_scores['CFT_mtld']

print("MTLD 점수 계산 완료. 결과 요약:")
print(MTLD_scores[['DPO_mtld', 'CFT_mtld', 'mtld_diff']].describe())

MTLD 계산을 위해 텍스트 전처리 중...
MTLD 점수 계산 중... (행이 많으면 시간이 걸릴 수 있습니다)
MTLD 점수 계산 완료. 결과 요약:
         DPO_mtld    CFT_mtld   mtld_diff
count  200.000000  200.000000  200.000000
mean    89.993840   85.609114    4.384726
std     22.951348   23.260316   24.166980
min     42.575734   39.607788  -72.153873
25%     75.259788   68.823967  -11.365269
50%     89.039708   85.231094    4.756503
75%    104.768275  100.294013   18.637026
max    169.666702  161.165745   94.787446


In [9]:
MTLD_scores = MTLD_scores.drop(columns=['DPO', 'CFT', 'Human'])
MTLD_scores.to_csv('essays_mtld_results.csv', index=False, encoding='utf-8-sig')

In [10]:
from scipy import stats

# NaN 값 제거 
paired_scores = MTLD_scores[['DPO_mtld', 'CFT_mtld']].dropna()

# Paired T-Test 실행
statistic, pvalue = stats.ttest_rel(paired_scores['DPO_mtld'], 
                                  paired_scores['CFT_mtld'])

print("\n--- MTLD (CFT vs DPO) ---")
print(f"  T-statistic: {statistic:.4f}")
print(f"  P-value: {pvalue:.6f}")

# 결과 해석
alpha = 0.05
if pvalue < alpha:
    print(f"\n결론: P-value({pvalue:.6f}) < {alpha} 이므로,")
    print("  'DPO'과 'CFT'의 어휘 다양성(MTLD) 차이는 통계적으로 유의미합니다.")
else:
    print(f"\n결론: P-value({pvalue:.6f}) >= {alpha} 이므로,")
    print("  'DPO'과 'CFT'의 어휘 다양성(MTLD) 차이는 유의미하지 않습니다.")

# 4. .describe() 결과와 비교
mean_diff = MTLD_scores['mtld_diff'].mean()
print(f"\n참고: 평균 MTLD 차이 (DPO - CFT): {mean_diff:.4f}")
if mean_diff > 0:
    print("  (평균적으로 'DPO' 텍스트의 MTLD 점수가 더 높습니다.)")
elif mean_diff < 0:
    print("  (평균적으로 'CFT' 텍스트의 MTLD 점수가 더 높습니다.)")


--- MTLD (CFT vs DPO) ---
  T-statistic: 2.5659
  P-value: 0.011027

결론: P-value(0.011027) < 0.05 이므로,
  'DPO'과 'CFT'의 어휘 다양성(MTLD) 차이는 통계적으로 유의미합니다.

참고: 평균 MTLD 차이 (DPO - CFT): 4.3847
  (평균적으로 'DPO' 텍스트의 MTLD 점수가 더 높습니다.)
